In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as color
import numpy as np

In [ ]:
ablty=20
coe=1+10/100

In [ ]:
# demandCoordinates = [(88, 16), (25, 76), (69, 13), (73, 56), (80, 100), (22, 92), (32, 84), (73, 46), (
#         29, 10), (92, 32), (44, 44), (55, 26), (71, 27), (51, 91), (89, 54), (43, 28), (40, 78), (60, 66)]
data=np.load("xy坐标.npy")
demandCoordinates=[(data[i][0],data[i][1]) for i in range(data.shape[0])]
size=data.shape[0]
q=[1 for i in range(size)]
p=300
for xy in demandCoordinates:
    plt.plot(xy[0],xy[1],c="b",marker="s")

In [ ]:
A=[(i,j) for i in range(size) for j in range(size)]
N=[i for i in range(size)]
dis={(i,j):np.hypot(demandCoordinates[i][0]-demandCoordinates[j][0],demandCoordinates[i][1]-demandCoordinates[j][1]) for i,j in A}

In [ ]:
from gurobipy import Model, GRB , quicksum

In [ ]:
mdl=Model("p-center")


In [ ]:
x=mdl.addVars(A,vtype=GRB.CONTINUOUS,ub=1,lb=0)
# x=mdl.addVars(A,vtype=GRB.BINARY)
n=mdl.addVars(N,vtype=GRB.INTEGER)

In [ ]:
mdl.ModelSense=GRB.MINIMIZE
mdl.setObjective(quicksum(x[i,j]*dis[i,j]*q[j] for i,j in A))

In [ ]:
mdl.addConstrs(quicksum(x[i,j] for j in N)== 1 for i in N);

In [ ]:
mdl.addConstr(quicksum(n[i] for i in N )== p);

In [ ]:
mdl.addConstrs((quicksum(q[i]*x[i,j] for i in N)<=n[j]*ablty )for j in N);

In [ ]:
mdl.optimize()

In [ ]:
# 画所有点及标注需求
markersize=3
linewidth=0.5
for i in N:
    xy=demandCoordinates[i]
    plt.plot(xy[0],xy[1],c="b",marker="s",markersize=markersize)
    #plt.text(xy[0],xy[1],f"{q[i]}")

# 画需求量
for i in N:
    if (n[i].x>0):
        xy=demandCoordinates[i]
        col=plt.cm.rainbow
        nMax=0.0
        for ni in N:
            # print(type(n[ni].x),n[ni].x,type(nMax))
            if n[ni].x>nMax:
                nMax=n[ni].x
        norm=color.Normalize(vmax=round(nMax)+1,vmin=0)
        plt.plot(xy[0],xy[1],marker="s",color=col(norm(n[i].x)),markersize=markersize)
        # plt.text(xy[0]-5,xy[1]-5,f"{n[i].x}",color="b")
        

# 画供应线
for i,j in A:
    if x[i,j].x>0.0:
        demandCoordinates[i][0],demandCoordinates[i][1]
        demandCoordinates[j][0],demandCoordinates[j][1]
        plt.plot([demandCoordinates[i][0],demandCoordinates[j][0]],[demandCoordinates[i][1],demandCoordinates[j][1]],color="r",linewidth=linewidth)
    

In [ ]:
for i in N:
    print(round(n[i].x),end=",")

In [ ]:
print(n[1].x)

In [ ]:
np.save()